In [17]:
########### SPECIFY WHICH TWO SITES/TASKS TO COMPARE ################

#this is a binary brain mask (same for all subject)
mask = "/om/group/saxelab/OpenAutism/Analysis/first_level_gorgolewski.py/CBT/sub-SAXCBTNT04/sholo/model/run1/MNI152NLin2009cAsym_brainmask.nii.gz"

smoothed = 0          # 0 (no aroma) or 1 (aroma)

# site 1 specs
site1 = 'tasti2'     # site is ID (e.g. SCOTTFM, CBT, DOD, tasti2, EIB)
task1 = 'sholo'       # task (e.g. sholo, morphing, etc.)
run1 = '001'          # run (e.g. 001 , 002, 003, 004)

save_path = "./Site_Variance_Plots/"
save_path2 = "./Site_Variance_Plots/Intrasite_Brains/"


In [18]:
from glob import glob
import re

if smoothed:
    aroma = '_variant-smoothAROMAnonaggr'
    variantpath = "/om/group/saxelab/OpenAutism/Analysis/aroma_bold_reference_images/"
else:
    aroma = ''
    variantpath = '/om/group/saxelab/OpenAutism/Analysis/bold_reference_images/'

# list all of the files in directory /om/group/saxelab/OpenAutism/Analysis/bold_reference_images
all_files = glob(variantpath+"*")
subject = []
count_1 = 0
count_2 = 0

# create list of subject in folder
for runs in range(0, len(all_files)):
    SITE1 = re.search('mean_sub-SAX'+site1+'(.+?)_task-'+task1+'_run-'+run1+'_bold_space-MNI152NLin2009cAsym'+aroma+'_preproc.nii.gz', all_files[runs])
    if SITE1:
        found = SITE1.group(1)
        if 'SAX'+site1+found+task1+run1 in open('Runs_MotionFiltered.csv').read():
            found = 'mean_sub-SAX'+site1+found+'_task-'+task1+'_run-'+run1+'_bold_space-MNI152NLin2009cAsym'+aroma+'_preproc.nii.gz'
            subject.append(found)
            count_1 = count_1 + 1
        else:
            print('EXCLUDE_'+site1+'_'+found)
subject.sort()

#print('Total= '+str(len(subject)))
#print(site1 + " = " +str(count_1))

EXCLUDE_tasti2_24
EXCLUDE_tasti2_41
EXCLUDE_tasti2_42
EXCLUDE_tasti2_48
Total= 41
tasti2 = 41


In [9]:
import os
import nibabel as nib
import numpy as np
from matplotlib.mlab import PCA


#result_array = np.array([])
result_array = np.empty((len(subject)-1,902629,)) #columns rows (MUST FIX TO THE SHAPE I NEED)--275275;325325
result_array[:] = np.nan
print(result_array.size)
largest_array = 0

for elements in range(0, (len(subject)-1)):

    #subj_ex = os.path.join('/om/group/saxelab/OpenAutism/Analysis/bold_reference_images/'+subject[elements])
    subj_ex = os.path.join(variantpath+subject[elements])
    
    Subject = nib.load(subj_ex)
    Subject_data = np.array(Subject.dataobj)
    #print(Subject_data.shape)
    
    Mask = nib.load(mask)
    Mask_data = np.array(Mask.dataobj)   
    
    Subject_data = Subject_data*Mask_data # added
    
    flat_subject = Subject_data.flatten()
    #print(flat_subject[1])
    #print(flat_subject.shape) 
    result_array[elements,:] = flat_subject
    
    if flat_subject.size > largest_array:
        largest_array = flat_subject.size       

print(result_array.shape)
print("Matrix = " + str(largest_array) + "x" + str(len(subject)))


87555013


FileNotFoundError: No such file: '/om/group/saxelab/OpenAutism/Analysis/bold_reference_images/mean_sub-SAXDOD001_task-morphing_run-001(.+?)_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'

In [4]:
# mean center the data (mixed opinions about this-mean centering seems dominant view)
a = result_array
to_norm = np.mean(a, axis=0) #uncommented
final = a - to_norm #uncommented

U, s, VT = np.linalg.svd(final, full_matrices=False) #changed to final

In [5]:
# as MURCIA et al did

All_Component_scores = U*s    #col by col

In [6]:
from nibabel.testing import data_path
from nilearn import plotting

# use temporarily
root = '/om/group/saxelab/OpenAutism/Analysis/ROIs/'
parcel_file = os.path.join(data_path, root + 'rSTS_FSL_space.nii.gz')
PARCEL = nib.load(parcel_file)


temp = VT[0,:]
temp2 = np.array(temp)
print(temp2.shape)

Eigenimage = np.reshape(temp2, (91,109,91))
#img = nib.Nifti1Image(Eigenimage, np.eye(4)) # use img
img = nib.Nifti1Image(Eigenimage, PARCEL.affine, PARCEL.header)
nib.save(img, save_path2+site1+task1+run1+'_Eigenimage_PC1.nii.gz')


# load in a nifti file and see if it works.
#temp_path = './Site_Variance_Plots/CBTsholo001_VS_DODmorphing001_Eigenimage_PC1.nii.gz' # use temp_path
temp_path = save_path2+site1+task1+run1+'_Eigenimage_PC1.nii.gz'

#plotting.plot_glass_brain(img, threshold=0.0001)
display = plotting.plot_glass_brain(temp_path, title='plot_glass_brain',
                          black_bg=True, display_mode='xz', threshold=0.00001)
#plotting.plot_connectome(img)

plotting.show()
  
#display.savefig(save_path2+site1+task1+run1+'_Eigenimage_PC1.png')
#plotting.plot_stat_map(IMG, output_file=save_path2+site1+task1+run1+'_Eigenimage_PC1.png') 
   
display.savefig(save_path2+site1+task1+run1+'_Eigenimage_PC1.png')
print(save_path2+site1+task1+run1+'_Eigenimage_PC1.png')

(902629,)


./Site_Variance_Plots/Intrasite_Brains/CBTsholo001_Eigenimage_PC1.png


In [ ]:
# apply binary mask to all brains * 